In [292]:
#!/usr/bin/python3
import numpy as np
import pandas as pd
from collections import Counter
from itertools import chain
from math import log
from sklearn.datasets import load_iris
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [293]:
def is_numeric(value):
    '''
    Helper function to check if value is numeric or not.
    '''
    return isinstance(value, int) or isinstance(value, float)

In [294]:
class SplittingCriteria:
    def __init__(self, column, value):
        '''
        This class initializes the column name and the value at each node where the split occurs.
        Eg - If questions is 'is sepal_length >= 1cm', then col=sepal_length and value=1cm
        '''
        self.column = column
        self.value = value

    def match(self, data):
        '''
        This function checks if the supplied data meets the splitting criteria posed by the Question.
        @returns - True if criteria is met, False otherwise
        '''
        value = data[self.column]
        if is_numeric(value):
            return value >= self.value
        else:
            return value == self.value
        
    def __repr__(self):
        '''
        This function prints the Question in a readable format.
        '''
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is {column} {condition} {value}?".format(column=self.column, condition=condition, value=self.value)

In [295]:
class DecisionNode:
    def __init__(self, split, true_branch, false_branch):
        '''
        This class represents the internal node structure of Decision Tree.
        @params - split: Stores column and value variables regarding the splitting criteria of that node
                  true_branch: points to the true branch after splitting
                  false_branch: points to the false branch after splitting
        '''
        self.split = split
        self.true_branch = true_branch
        self.false_branch = false_branch

In [296]:
class DecisionLeaf:
    def __init__(self, predictions):
        '''
        This class represents the leaf node structure of the Decision Tree.
        @params - predictions: A dictionary that stores the counts of occurences of the different classes
        '''
        self.predictions = predictions

In [297]:
class DecisionTree:
    def __init__(self, criterion='gini'):
        '''
        This class represents a Decision tree along with the required methods.
        @params - criterion: Criteria to be used to measure quality of split.
                             Supported criteria are 'gini' for gini impurity and 
                             'entropy' for information gain.
        '''
        
        def gini(dataset):
            '''
            This function calculates the Gini impurity of the given dataset.
            @params - dataset: The dataset whose impurity is to be calculated
            @returns - impurity: The impurity value of the data
            '''
            count = self.label_counts(dataset)
            impurity = 1
            for label in count:
                probability_of_label = count[label] / float(len(dataset))
                impurity -= probability_of_label**2
            return impurity
    
        def entropy(dataset):
            '''
            This function calculates the Entropy of the given dataset.
            @params - dataset: The dataset whose entropy is to be calculated
            @returns - entropy: The entropy value for the given dataset
            '''
            count = self.label_counts(dataset)
            entropy = 0
            for label in count:
                probability_of_label = count[label] / float(len(dataset))
                entropy -= probability_of_label * log(probability_of_label, 2)
            return entropy
        
        criteria = {'gini':gini, 'entropy':entropy}
        self.evaluation_function = criteria.get(criterion, gini)

    def partition(self, dataset, splitting_criteria):
        '''
        This function partitions the dataset based on the given splitting criteria.
        @params - dataset: The dataset to be split
                  splitting_criteria: The column and value on which the split needs to be performed
        @returns - true_dataset: The records in the dataset that satisfy the criteria
                   false_dataset: The records in the dataset that do not satisfy the criteria
        '''
        true_dataset, false_dataset = [], []
        for row in dataset:
            if splitting_criteria.match(row):
                true_dataset.append(row)
            else:
                false_dataset.append(row)
        return true_dataset, false_dataset
    
    def label_counts(self, dataset):
        '''
        This function that counts the number of instances for each label in the dataset.
        @params - row_labels: The labels for each row in the dataset
        @returns - count: A dictionary where keys are the different labels and the corresponding value
                          is the frequency of occurence of the label
        '''
        count={}
        #takes whole dataset in as argument
        for row in dataset:
            #traverse on each datapoint
            label=row[-1]
            #labels are in the last column
            #if label is not even once come initialise it
            if label not in count:
                count[label]=0
            #increase the count of present label by 1
            count[label]+=1
        return count 
    
    def information_gain(self, current, left, right):
        '''
        This function calculates the information gain using the evaluation function.
        @params - current: The gini impurity or entropy value of the current full dataset
                  left: The true branch after splitting
                  right: The false branch after splitting
        @returns - info_gain: The information_gain value at the current node
        '''
        p = float(len(left) / (len(left) + len(right)))
        info_gain = current - p * self.evaluation_function(left) - (1 - p) * self.evaluation_function(right)
        return info_gain
    
    def find_best_split(self, dataset):
        '''
        This function finds the column that provides the best information gain and the splitting value.
        @params - dataset: The dataset in which the best split needs to be found
        @returns - best_gain: The best gain value using the current dataset
                   splitting_criteria: The splitting criteria that contains the splitting column and value
        '''
        best_gain = 0
        best_splitting_criteria = None
        # Calculate the current gain
        current_gain = self.evaluation_function(dataset)
        # The total number of features without the target
        n_features = len(dataset[0]) - 1
        for col in range(n_features):
            # Set of all unique values in a single feature
            feature_values = set([row[col] for row in dataset])
            # Iterate through each unique class and try to split using that value
            for value in feature_values:
                # Create the splitting criteria
                splitting_criteria = SplittingCriteria(col, value)
                # Devide the dataset based on the criteria
                true_dataset, false_dataset = self.partition(dataset, splitting_criteria)
                # No splitting occurs, let us go to the next iteration
                if len(true_dataset) == 0 or len(false_dataset) == 0:
                    continue
                info_gain = self.information_gain(current_gain, true_dataset, false_dataset)
                # Check if it is more than the best gain yet
                if info_gain > best_gain:
                    best_gain, best_splitting_criteria = info_gain, splitting_criteria
        return best_gain, best_splitting_criteria
    
    def build_tree(self, dataset):
        '''
        This function builds the actual Decision Tree.
        @params - dataset: The training dataset
        @returns - root_node: The root node of the Decision tree
        '''
        # take the whole dataset and find the initial best split
        info_gain, splitting_criteria = self.find_best_split(dataset)
        # If info_gain is zero, leaf node condition is satisfied
        if info_gain == 0:
            return DecisionLeaf(self.label_counts(dataset))
        # Else, we have found the best splitting criteria
        true_dataset, false_dataset = self.partition(dataset, splitting_criteria)
        # Recursively build the tree for the true and false branches
        true_branch = self.build_tree(true_dataset)
        false_branch = self.build_tree(false_dataset)
        # Return the root node of the tree
        return DecisionNode(splitting_criteria, true_branch, false_branch)

In [298]:
class CustomDecisionTreeClassifier:
    def __init__(self, criterion='gini'):
        '''
        This class creates a classifier using the Decision Tree class.
        @params - criterion: Criteria to be used to measure quality of split.
                             Supported criteria are 'gini' for gini impurity and 
                             'entropy' for information gain.
        '''
        self.criterion = criterion
    
    def check_input(self, X, y=None):
        '''
        Checks if the input is a Pandas DataFrame, Pandas Series or Numpy array.
        @params - X: Data
                  y: Target values         
        @returns - False if input is not valid,
                   True otherwise
        '''
        if not isinstance(X, (pd.Series, pd.DataFrame, np.ndarray)):
            error = """ X required as a Pandas DataFrame, 
                        Pandas Series or a Numpy array. Found type: {datatype_x}
                    """.format(datatype_x=type(X))
            raise TypeError(error)
            return False
        if y is not None and not isinstance(y, (pd.Series, pd.DataFrame, np.ndarray)):
            error = """ y required as a Pandas DataFrame, 
                        Pandas Series or a Numpy array. Found type: {datatype_y}
                    """.format(datatype_y=type(y))
            raise TypeError(error)
            return False
        return True
    
    def transform_input(self, X, y=None):
        '''
        Transforms the input into 2D numpy array before fitting the model.
        @params - X: Data
                  y: Target values      
        @returns: If input is of valid format:
                         X: Returns data X as a 2D numpy array
                         y: Returns target y as a 2D numpy array
                  Otherwise returns the input as it is
        '''
        if self.check_input(X, y) is True:
            # Convert X into numpy array
            if isinstance(X, pd.DataFrame):
                X = X.values
            elif isinstance(X, pd.Series):
                X = X.values
                X = X[:, None]
            # Convert y into numpy array
            if y is not None:
                if isinstance(y, pd.DataFrame):
                    y = y.values
                elif isinstance(y, pd.Series):
                    y = y.values
                    y = y[:, None]
        if y is None:
            return X
        else:
            return X, y
    
    def fit(self, X, y):
        '''
        This function uses the given dataset and creates a decision tree.
        @params - X: Training data
                  y: Labels of training data
        '''
        X, y = self.transform_input(X, y)
        dataset = np.c_[X, y]
        dt = DecisionTree(criterion=self.criterion)
        self.tree_ = dt.build_tree(dataset)
        return self
    
    def predict(self, X):
        '''
        This function predicts the labels of the given data.
        @params - X: Data for which labels have to be predicted
        @returns - y: List of predicted labels
        '''
        if self.tree_ is None:
            print("Please fit the dataset using the fit function before prediciting values")
            return
        X = self.transform_input(X)
        y = [self.predictInstance(row) for row in X]
        return np.array(y).astype(int)
    
    def predictInstance(self, row):
        '''
        This function recursively iterates through the decision tree and predicts the label
        of a single row of data.
        @params - row: A single row of data
        @returns - row_pred: The label predicted by the decision tree for this data
        '''
        dt_node = self.tree_
        while type(dt_node) is not DecisionLeaf:
            splitting_criteria = dt_node.split
            if splitting_criteria.match(row):
                dt_node = dt_node.true_branch
            else:
                dt_node = dt_node.false_branch
        return list(dt_node.predictions.keys())[0]
            

In [299]:
# Loading the dataset

dataset = pd.read_csv( 'dataset.csv',\
                        names=['ID', 'Age', 'Income', 'Gender', 'Marital Status', 'Buys'],\
                        skiprows=1 )
# Drop the ID column as Pandas provides indices by default
dataset = dataset.drop('ID', axis=1)
dataset.head()

,Age,Income,Gender,Marital Status,Buys
0,< 21,High,Male,Single,No
1,< 21,High,Male,Married,No
2,21-35,High,Male,Single,Yes
3,> 35,Medium,Male,Single,Yes
4,> 35,Low,Female,Single,Yes


In [300]:
# Load some more info about the dataset before proceeding

dataset.info()
dataset.head(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 5 columns):
Age               14 non-null object
Income            14 non-null object
Gender            14 non-null object
Marital Status    14 non-null object
Buys              14 non-null object
dtypes: object(5)
memory usage: 640.0+ bytes


,Age,Income,Gender,Marital Status,Buys
0,< 21,High,Male,Single,No
1,< 21,High,Male,Married,No
2,21-35,High,Male,Single,Yes
3,> 35,Medium,Male,Single,Yes
4,> 35,Low,Female,Single,Yes
5,> 35,Low,Female,Married,No
6,21-35,Low,Female,Married,Yes
7,< 21,Medium,Male,Single,No
8,< 21,Low,Female,Married,Yes
9,> 35,Medium,Female,Single,Yes


In [301]:
# Encode the values in the different columns
def encode_dataset(dataset):
    # 1. Encoding the Age column
    # We can see that there are three different categories of ages:
    # i. < 21
    # ii. 21-35
    # iii. > 35
    # We will treat these as ordinal values and encode them
    dataset['Age'] = dataset['Age'].map({'< 21':0, '21-35':1, '> 35':2})

    # 2. Encoding the Income column
    # Again, there are 3 different categories of incomes:
    # i. Low
    # ii. Medium
    # iii. High
    # We will again use the respective ordinal values to encode them
    dataset['Income'] = dataset['Income'].map({'Low':0, 'Medium':1, 'High':2})

    # 3. Encoding the Gender column
    # The Gender column has two categories
    # i. Female = 0
    # ii. Male = 1
    dataset['Gender'] = dataset['Gender'].map({'Female':0, 'Male':1})

    # 4. Encoding the marital status
    # Treat this similar to Gender
    # i. Single = 0
    # ii. Married = 1
    dataset['Marital Status'] = dataset['Marital Status'].map({'Single':0, 'Married':1})

    # 4. Encoding the target column
    # Again, this too is a binary column
    # i. No = 0
    # ii. Yes = 1
    if 'Buys' in dataset.columns: # Target column may or may not be part of dataframe
        dataset['Buys'] = dataset['Buys'].map({'No':0, 'Yes':1})

In [302]:
encode_dataset(dataset)
dataset.head(20)

,Age,Income,Gender,Marital Status,Buys
0,0,2,1,0,0
1,0,2,1,1,0
2,1,2,1,0,1
3,2,1,1,0,1
4,2,0,0,0,1
5,2,0,0,1,0
6,1,0,0,1,1
7,0,1,1,0,0
8,0,0,0,1,1
9,2,1,0,0,1


In [303]:
# Split the dataset into training and testing
X = dataset.drop('Buys', axis=1)
y = dataset['Buys']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.44, random_state=100)

In [304]:
# Classification using Custom Decision Tree

custom_classifier = CustomDecisionTreeClassifier()
y_pred = custom_classifier.fit(X_train, y_train).predict(X_test)
score = accuracy_score(y_pred, y_test)
print("Running custom decision tree class...")
print("Root of decision tree: {root}".format(root=custom_classifier.tree_.split))
print("Predicted values: {y_pred}".format(y_pred=y_pred))
print("Testing accuracy_score: {score}".format(score=score))

Running custom decision tree class...
Root of decision tree: Is 2 == 0?
Predicted values: [0 1 1 0 1 1 1]
Testing accuracy_score: 0.7142857142857143


In [305]:
# Classification using Sklearn Decision Tree

sklearn_classifier = tree.DecisionTreeClassifier()
y_pred = sklearn_classifier.fit(X_train, y_train).predict(X_test)
score = accuracy_score(y_pred, y_test)
print("Running sklearn decision tree class...")
print("Root of decision tree: {root}".format(root=tree.export_graphviz(sklearn_classifier)[44:55]))
print("Predicted values: {y_pred}".format(y_pred=y_pred))
print("Testing accuracy_score: {score}".format(score=score))

Running sklearn decision tree class...
Root of decision tree: X[2] <= 0.5
Predicted values: [0 1 1 0 1 1 1]
Testing accuracy_score: 0.7142857142857143


In [330]:
# Encode following test data
# test data: [Age < 21, Income = Low, Gender = Female, Marital Status = Married]

data = [["< 21", "Low", "Female", "Married"]]
df = pd.DataFrame(data, columns=['Age', 'Income', 'Gender', 'Marital Status'])
encode_dataset(df)
df.head()

,Age,Income,Gender,Marital Status
0,0,0,0,1


In [331]:
# Predict using custom classifier

y_pred = custom_classifier.predict(df)
if y_pred:
    y_pred = "Yes"
else:
    y_pred = "No"
print("Prediction of custom classifier: {y_pred}".format(y_pred=y_pred))

Prediction of custom classifier: Yes


In [332]:
# Predict using custom classifier

y_pred = sklearn_classifier.predict(df)
if y_pred:
    y_pred = "Yes"
else:
    y_pred = "No"
print("Prediction of sklearn classifier: {y_pred}".format(y_pred=y_pred))

Prediction of sklearn classifier: Yes
